In [2]:
import numpy as np
import pandas as pd
# Define the input data (15 days of sales, 1 value per day)
data = np.array([100, 90, 120, 80, 150, 110, 130, 140, 115, 105, 90, 125, 135, 100, 95])
#data = np.arange(100000)

In [6]:
np.log(0)

/tmp/ipykernel_2146/2933082444.py:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf

In [32]:
# Define the initial sales data as a list of numbers
data = [10, 12, 15]

# Define the mean and standard deviation of the initial sales data
mean_sales = np.mean(data)
std_sales = np.std(data)

# Define a function to generate new sales data based on the last 3 days
def generate_new_sales(last_sales):
    # Calculate the mean and standard deviation of the last 3 days of sales
    last_mean = np.mean(last_sales)
    last_std = np.std(last_sales)

    # Calculate the new sales value as the mean of the last 3 days plus a random amount
    value = np.round(last_mean+(last_mean*0.12))
    return value

# Generate the next 10 days of sales data based on the initial sales data
for i in range(51):
    # Get the last 3 days of sales data
    last_sales = data[-3:]

    # Generate the new sales data based on the last 3 days
    new_sales = generate_new_sales(last_sales)

    # Append the new sales data to the initial sales list
    data.append(new_sales)

In [33]:
df = pd.DataFrame(data, columns=['Ice Cream Sales'])
mean = np.mean(data)
std = np.std(data)

# Normalize the array along the columns
data = (data - mean) / std

df['diff'] = df['Ice Cream Sales'].diff()
data = df['diff'].values[1:]
df = pd.DataFrame(data, columns=['Diff Sales'])

mean = np.mean(data)
std = np.std(data)

# Normalize the array along the columns
data = (data - mean) / std
df = pd.DataFrame(data, columns=['Ice Cream Sales'])

In [35]:



mean = np.mean(data)
std = np.std(data)

# Normalize the array along the columns
data = (data - mean) / std
# Define the number of time steps in the input sequence
num_time_steps = 3

# Define the number of input features (in this case, we only have 1 input feature: sales)
num_input_features = 1

# Define the number of hidden units in the RNN layer
num_hidden_units = 5

# Define the batch size
batch_size = 10

# Initialize the weight matrices and bias vectors for the RNN layer
W_xh = np.random.randn(num_input_features, num_hidden_units)
W_hh = np.random.randn(num_hidden_units, num_hidden_units)
b_h = np.zeros((1, num_hidden_units))
W_hy = np.random.randn(num_hidden_units, num_input_features)
b_y = np.zeros((1, num_input_features))

# Initialize the hidden state (this is the initial state before processing any input)
h_t = np.zeros((batch_size, num_hidden_units))

# Define the learning rate
learning_rate = 0.001

for each in range(10000):
# Loop through the input sequence and update the hidden state at each time step
    for i in range(0, len(data) - num_time_steps, batch_size):
        # Extract the input sequence for the current batch
        x_batch = np.zeros((batch_size, num_time_steps, num_input_features))
        for j in range(batch_size):
            x_batch[j,:,:] = data[i+j:i+j+num_time_steps].reshape((num_time_steps, num_input_features))
        
        # Compute the new hidden states for the current batch using the current inputs and previous hidden states
        h_t = np.zeros((batch_size, num_hidden_units))
        for t in range(num_time_steps):
            x_t = x_batch[:,t,:]
            h_t = np.tanh(np.dot(x_t, W_xh) + np.dot(h_t, W_hh) + b_h)
        
        # Compute the outputs for the current batch at the last time step
        y_pred = np.dot(h_t, W_hy) + b_y
        
        # Compute the target values for the current batch (in this case, we want to predict the next day's sales based on the previous 3 days of sales)
        y_true = data[i+num_time_steps:i+num_time_steps+batch_size].reshape((batch_size, num_input_features))
        
        # Compute the error (mean squared error) for the current batch
        error = 0.5 * np.mean((y_pred - y_true) ** 2)
        
        # Compute the gradients of the output layer for the current batch (using the chain rule)
        grad_y = (y_pred - y_true) / batch_size
        grad_W_hy = np.dot(h_t.T, grad_y)
        grad_b_y = np.sum(grad_y, axis=0, keepdims=True)
        
        # Initialize the gradients of the hidden state for the current batch (this will be used as the initial gradients for backpropagation)
        grad_h = np.zeros((batch_size, num_hidden_units))
        
        # Loop backward through the time steps and compute the gradients for each time step for the current batch
        for t in reversed(range(num_time_steps)):
            x_t = x_batch[:,t,:]
            h_t = np.tanh(np.dot(x_t, W_xh) + np.dot(h_t, W_hh) + b_h)
            
            # Compute the gradients for the output of the RNN layer (using the chain rule)
            grad_output = grad_h + np.dot(grad_y, W_hy.T)
            grad_z = grad_output * (1 - h_t ** 2)
            
            # Compute the gradients for the parameters of the RNN layer (using the chain rule)
            grad_W_xh = np.dot(x_t.T, grad_z)
            grad_W_hh = np.dot(h_t.T, grad_z)
            grad_b_h = np.sum(grad_z, axis=0, keepdims=True)
            
            # Update the gradients for the next time step (using the chain rule)
            grad_h = np.dot(grad_z, W_hh.T)
            
            # Accumulate the gradients for the current batch
            if t == num_time_steps - 1:
                total_grad_W_xh = grad_W_xh
                total_grad_W_hh = grad_W_hh
                total_grad_b_h = grad_b_h
                total_grad_W_hy = grad_W_hy
                total_grad_b_y = grad_b_y
            else:
                total_grad_W_xh += grad_W_xh
                total_grad_W_hh += grad_W_hh
                total_grad_b_h += grad_b_h
                total_grad_W_hy += grad_W_hy
                total_grad_b_y += grad_b_y

        # Update the parameters of the RNN layer for the current batch using the computed gradients and the learning rate
        W_xh -= learning_rate * total_grad_W_xh
        W_hh -= learning_rate * total_grad_W_hh
        b_h -= learning_rate * total_grad_b_h
        W_hy -= learning_rate * total_grad_W_hy
        b_y -= learning_rate * total_grad_b_y
        print("Error:", error)

Error: 0.148589192880945
Error: 0.11523166006765997
Error: 0.08657645286275317
Error: 0.18651502605647782
Error: 0.14299306393006755
Error: 0.14541199043780478
Error: 0.11092036733095585
Error: 0.08296063427663553
Error: 0.1814603419017354
Error: 0.13349578127116657
Error: 0.1426761572374952
Error: 0.10709335614370277
Error: 0.07971189896033638
Error: 0.17668261739755117
Error: 0.12475060624333756
Error: 0.14032794614625174
Error: 0.10369636883898088
Error: 0.07678966115091879
Error: 0.17216674969566864
Error: 0.11669939717394524
Error: 0.13831976652728512
Error: 0.10068127362977856
Error: 0.0741580746043399
Error: 0.16789831952689938
Error: 0.1092882382737532
Error: 0.1366094778980925
Error: 0.09800535164869308
Error: 0.07178544640102114
Error: 0.16386357767603763
Error: 0.10246717542826833
Error: 0.1351597677339974
Error: 0.0956306712373662
Error: 0.06964372799192158
Error: 0.1600494293493403
Error: 0.09618996125633225
Error: 0.13393760261983192
Error: 0.09352353793744832
Error: 0.06

In [5]:
for each in range(0, len(data) - num_time_steps, batch_size):
    print(each)

0
4
8


In [1]:
np.log(1)

NameError: name 'np' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, Dense
from tensorflow.keras.models import Model

# Define the input shape and parameters of the RNN layer
num_time_steps = 3
num_input_features = 1
num_hidden_units = 5

# Define the input layer
input_layer = Input(shape=(num_time_steps, num_input_features))

# Define the RNN layer
rnn_layer = SimpleRNN(units=num_hidden_units, activation='tanh')

# Define the output layer
output_layer = Dense(units=1)

# Connect the input layer to the RNN layer
rnn_output = rnn_layer(input_layer)

# Connect the RNN layer to the output layer
output = output_layer(rnn_output)

# Define the model
model = Model(inputs=input_layer, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Generate some random data
x = np.random.randn(100, num_time_steps, num_input_features)
y = np.random.randn(100, 1)

# Train the model
model.fit(x, y, epochs=10)


In [9]:
1- 0.000001**2

0.999999999999